In [29]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/breast_cancer_survival[1].csv")
print(data.head())

   Age  Gender  Protein1  Protein2  Protein3  Protein4 Tumour_Stage  \
0   42  FEMALE   0.95256   2.15000  0.007972 -0.048340           II   
1   54  FEMALE   0.00000   1.38020 -0.498030 -0.507320           II   
2   63  FEMALE  -0.52303   1.76400 -0.370190  0.010815           II   
3   78  FEMALE  -0.87618   0.12943 -0.370380  0.132190            I   
4   42  FEMALE   0.22611   1.74910 -0.543970 -0.390210           II   

                       Histology ER status PR status HER2 status Surgery_type  \
0  Infiltrating Ductal Carcinoma  Positive  Positive    Negative        Other   
1  Infiltrating Ductal Carcinoma  Positive  Positive    Negative        Other   
2  Infiltrating Ductal Carcinoma  Positive  Positive    Negative   Lumpectomy   
3  Infiltrating Ductal Carcinoma  Positive  Positive    Negative        Other   
4  Infiltrating Ductal Carcinoma  Positive  Positive    Positive   Lumpectomy   

  Date_of_Surgery Date_of_Last_Visit Patient_Status  
0       20-May-18          26-Au

In [31]:
print(data.dtypes)
print("shape",data.shape)

Age                     int64
Gender                 object
Protein1              float64
Protein2              float64
Protein3              float64
Protein4              float64
Tumour_Stage           object
Histology              object
ER status              object
PR status              object
HER2 status            object
Surgery_type           object
Date_of_Surgery        object
Date_of_Last_Visit     object
Patient_Status         object
dtype: object
shape (334, 15)


In [36]:
h=data.drop({'Histology','ER status','PR status','Date_of_Surgery','Date_of_Last_Visit','Surgery_type','HER2 status'},axis=1)
print(h.isnull().sum())
data['Patient_Status'].isnull().sum()/h.shape[0]

Age                0
Gender             0
Protein1           0
Protein2           0
Protein3           0
Protein4           0
Tumour_Stage       0
Patient_Status    13
dtype: int64


0.038922155688622756

In [37]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
encoder = LabelEncoder()
h['Gender'] = encoder.fit_transform(h['Gender'])
# Define mapping for 'Gender' column
gender_mapping = {'Female': 1, 'Male': 0}

# Map 'Gender' column using the defined mapping, and fill any NaN values with 0
h['Gender'] = h['Gender'].map(gender_mapping).fillna(0).astype(int)

# Define mapping for 'Tumour_Stage' column
tumour_stage_mapping = {'Stage I': 1, 'Stage II': 2, 'Stage III': 3}

# Map 'Tumour_Stage' column using the defined mapping, and fill any NaN values with 0
h['Tumour_Stage'] = h['Tumour_Stage'].map(tumour_stage_mapping).fillna(0).astype(int)
Patient_Status_mapping = {'Alive': 1, 'Dead': 0}


h['Patient_Status'] = h['Patient_Status'].map(Patient_Status_mapping).fillna(0).astype(int)

# Now, 'Gender' column will have 1 for 'Female' and 0 for 'Male', and 'Tumour_Stage' column will have corresponding integers for each stage.


h.describe()

,Age,Gender,Protein1,Protein2,Protein3,Protein4,Tumour_Stage,Patient_Status
count,334.000000,334.0,334.000000,334.000000,334.000000,334.000000,334.0,334.000000
mean,58.886228,0.0,-0.029991,0.946896,-0.090204,0.009819,0.0,0.763473
std,12.961212,0.0,0.563588,0.911637,0.585175,0.629055,0.0,0.425587
min,29.000000,0.0,-2.340900,-0.978730,-1.627400,-2.025500,0.0,0.000000
25%,49.000000,0.0,-0.358888,0.362173,-0.513748,-0.377090,0.0,1.000000
50%,58.000000,0.0,0.006129,0.992805,-0.173180,0.041768,0.0,1.000000
75%,68.000000,0.0,0.343598,1.627900,0.278353,0.425630,0.0,1.000000
max,90.000000,0.0,1.593600,3.402200,2.193400,1.629900,0.0,1.000000


In [38]:
y=h['Patient_Status']
x=h.drop('Patient_Status',axis=1)
x=x.fillna(0)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)
print(x_train.isnull())

print(x_train)

       Age  Gender  Protein1  Protein2  Protein3  Protein4  Tumour_Stage
224  False   False     False     False     False     False         False
78   False   False     False     False     False     False         False
295  False   False     False     False     False     False         False
17   False   False     False     False     False     False         False
24   False   False     False     False     False     False         False
..     ...     ...       ...       ...       ...       ...           ...
188  False   False     False     False     False     False         False
71   False   False     False     False     False     False         False
106  False   False     False     False     False     False         False
270  False   False     False     False     False     False         False
102  False   False     False     False     False     False         False

[267 rows x 7 columns]
     Age  Gender  Protein1  Protein2  Protein3  Protein4  Tumour_Stage
224   38       0 -0.268450   

In [41]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(x_train)
X_test_scaled = scaler.transform(x_test)

In [42]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
gboost_model = GradientBoostingClassifier()
gboost_model.fit(X_train_scaled, y_train)
gboost_pred = gboost_model.predict(X_test_scaled)
gboost_accuracy = accuracy_score(y_test, gboost_pred)
gboost_precision = precision_score(y_test, gboost_pred)
gboost_recall = recall_score(y_test, gboost_pred)
gboost_f1 = f1_score(y_test, gboost_pred)

print("Gradient Boosting Classifier:")
print("Accuracy:", gboost_accuracy)
print("Precision:", gboost_precision)
print("Recall:", gboost_recall)
print("F1-score:",gboost_f1)

Gradient Boosting Classifier:
Accuracy: 0.746268656716418
Precision: 0.7692307692307693
Recall: 0.9615384615384616
F1-score: 0.8547008547008548


In [43]:
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train_scaled, y_train)
xgb_pred = xgb_model.predict(X_test_scaled)
xgb_accuracy = accuracy_score(y_test, xgb_pred)
xgb_precision = precision_score(y_test, xgb_pred)
xgb_recall = recall_score(y_test, xgb_pred)
xgb_f1 = f1_score(y_test, xgb_pred)

print("\nXGBoost Classifier:")
print("Accuracy:", xgb_accuracy)
print("Precision:", xgb_precision)
print("Recall:", xgb_recall)
print("F1-score:",xgb_f1)


XGBoost Classifier:
Accuracy: 0.7014925373134329
Precision: 0.7580645161290323
Recall: 0.9038461538461539
F1-score: 0.8245614035087719
